<a href="https://colab.research.google.com/github/VeereshElango/ai_powered_automated_tamil_song_lyrics_generator/blob/master/AutomatedTamilSongGenerationUsingDeepLearning_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutomatedTamilSongGenerationUsingDeepLearning

This notebook was built on top of Tensorflow's tutorial [Text generation with an RNN](https://www.tensorflow.org/tutorials/text/text_generation). The data utilized in this notebook can be find [here](https://cutt.ly/tt7krlz).

**Updates in this version:**


*   Added lyricist specific model Vairamuthu, Kannadasan & Vaali
*   Added evaluation of unigram and bigram pairs on generated lyrics at different diversities



## Data Import & Cleaning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

In [0]:
df = pd.read_csv("/content/drive/My Drive/ColabDatasets/Public/tamil_song_lyrics/tamilsonglyrics.csv", index_col=[0])

In [4]:
df= df.sample(frac=1).reset_index(drop=True)
df.head()

,Lyricist,Lyrics
0,Thamarai,கடலினில் மீனாக இருந்தவள் நான்\r\nஉனக்கென கரை த...
1,Vairamuthu,சிக்கு புக்கு சிக்கு புக்கு ரயிலே அட கலக்குது ...
2,NaN,உனைக் கண்ட கனவொன்று\r\nதேன் போலே\r\nஎனக்குயிரூ...
3,NaN,நடப்பது அறுபத்தெட்டுஇது அறுபத்தெட்டு\r\nஇன்னும...
4,Na. Muthukumar,ரப்பப்பபா… ரபிப்பா… ரப்பப்பா… ரபிப்பா…\r\nரப்ப...


In [5]:
df.Lyricist.value_counts().head(10)

Vairamuthu        542
Vaali             487
Kannadasan        290
Na. Muthukumar    265
Madhan Karky      136
Pa. Vijay         135
Yugabharathi      130
Thamarai           87
Viveka             74
Gangai Amaran      69
Name: Lyricist, dtype: int64

## Data Preparation

In [0]:
# Importing the libraries
import tensorflow as tf
import numpy as np
import os
import time

In [0]:
%%capture
! pip install tensorflowjs

In [0]:
import json
import tensorflowjs as tfjs

### Let's create a model for top 3 lyricists

Check for duplicates lyricist names and remove them if any

In [0]:
df.Lyricist = df.Lyricist.apply(lambda row: str(row).lower())

**Vairamuthu**

In [10]:
variamuthu_df = df[df.Lyricist.str.contains("vairamuthu")]
print("Total songs in dataset for Variamuthu: ", len(variamuthu_df))

Total songs in dataset for Variamuthu:  542


**Vaali**

In [11]:
vaali_df = df[df.Lyricist.str.contains("vaali")]
print("Total songs in dataset for Variamuthu: ", len(vaali_df))

Total songs in dataset for Variamuthu:  493


**Kannadasan**

In [12]:
kannadasan_df = df[df.Lyricist.str.contains("kannadasan")]
print("Total songs in dataset for Variamuthu: ", len(kannadasan_df))

Total songs in dataset for Variamuthu:  290


## Words Collection

In [0]:
import nltk
from nltk.util import ngrams

In [14]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
def word_collection(song_lyric, ngram_pair=(1,)):
    words = []
    for sentence in song_lyric.split("\n"):
        tokens = nltk.word_tokenize(sentence) #[ t for t in sentence.strip().split() if len(t)!=0]
        if 1 in ngram_pair:
            words.extend(tokens)
        if 2 in ngram_pair:
            bigrams = [ " ".join(bigram) for bigram in ngrams(tokens, 2)]
            words.extend(bigrams)
    return words
#Test
abc="""
+யாரோ யார் இவன்
யாரோ யார் இவன்
"""
out = word_collection(abc,(2,))
print(out)

['+யாரோ யார்', 'யார் இவன்', 'யாரோ யார்', 'யார் இவன்']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [16]:
WORDS_COLLECTION = []
for lyric in df.Lyrics.tolist():
    words = word_collection(lyric,(1,2))
    WORDS_COLLECTION.extend(words)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [17]:
print("Total words with Unigram & Bigram: ",len(WORDS_COLLECTION))

Total words with Unigram & Bigram:  1288975


In [18]:
WORDS_COLLECTION[:10]

['கடலினில்',
 'மீனாக',
 'இருந்தவள்',
 'நான்',
 'கடலினில் மீனாக',
 'மீனாக இருந்தவள்',
 'இருந்தவள் நான்',
 'உனக்கென',
 'கரை',
 'தாண்டி']

## Model Creation

### Evaluation

In [0]:
def generate_text(model, char2idx, idx2char, start_string, temperature=1.0, num_generate = 500):
  # Evaluation step (generating text using the learned model)

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temperature

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)
      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature 
      cat_dist_predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([cat_dist_predicted_id], 0)
      character_generated = idx2char[cat_dist_predicted_id]
      text_generated.append(character_generated)
      
  return (start_string + ''.join(text_generated))

In [0]:
def get_score(generated_text, ngram_value, words_collection):
    total_matched_words = 0
    unmatched_words= []
    gen_words = word_collection(generated_text, ngram_value)
    for word in gen_words:
        if word in words_collection:
            total_matched_words+=1
        else:
            unmatched_words.append(word)    
    score = round(total_matched_words/len(gen_words),3)
    return score, unmatched_words

def evaluate_single_song(generated_text, words_collection =WORDS_COLLECTION):
    uni_score, uni_unmatched_words = get_score(generated_text, (1,), words_collection)
    bi_score, bi_unmatched_words = get_score(generated_text, (2,), words_collection)
    unmatched_words = uni_unmatched_words + bi_unmatched_words
    return uni_score,bi_score, unmatched_words

def evaluate_model(model, model_name, char2idx, idx2char, debug=False):
    diveristy_values = [i for i in np.arange(0.1, 1, 0.1)]
    uni_score_col = []
    bi_score_col = []
    generated_texts_col = []
    unmatched_words_col = []
    for diversity in diveristy_values:
        generated_text = generate_text(model, char2idx, idx2char, "யாரோ", diversity)
        uni_score, bi_score, unmatched_words = evaluate_single_song(generated_text)
        uni_score_col.append(uni_score)
        bi_score_col.append(bi_score)
        generated_texts_col.append(generated_text)
        unmatched_words_col.append(unmatched_words)
        if debug:
            print("Model "+model_name+" - Diversity "+str(diversity)+" - Uni score "+str(uni_score)+" - Bi score "+str(bi_score))
    return pd.DataFrame(
        {
            model_name+"_uni_score":uni_score_col, 
            model_name+"_bi_score":bi_score_col,
            model_name+"_generated_lyric":generated_texts_col,
            model_name+"_unmatched_words":unmatched_words_col
         }
         )

In [21]:
# Sample Evaluation
abc="""
+யாரோ யார் இவன்
யாரோ யார் இவன்

தாயின் க
தானாய் உதித்தவன்
"""
uni_score, bi_score, unmatched_words = evaluate_single_song(abc, WORDS_COLLECTION)
print(uni_score, bi_score, unmatched_words)

0.9 0.667 ['+யாரோ', '+யாரோ யார்', 'தாயின் க']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: generator 'ngrams' raised StopIteration
  


### Model Design

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


In [0]:
def create_model(model_name, df, path_to_save_model, callbacks, build_model_func, checkpoint_dir, EPOCHS=30):
    print("Creating Model: ", model_name)
    print("-------------------------------")
    # Concatenating the strings
    text = " ".join([ song.strip() for song in df.Lyrics])
    print("Total songs combined : {}".format(len(df.Lyrics)))
    print()
    print ('Length of text: {} characters'.format(len(text)))
    print("Sample Text: ")
    print()
    print(text[:250])
    print()
    vocab = sorted(set(text))
    print ('{} unique characters'.format(len(vocab)))

    # Creating a mapping from unique characters to indices
    char2idx = {u:i for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)

    text_as_int = np.array([char2idx[c] for c in text])

    # The maximum length sentence we want for a single input in characters
    seq_length = 100
    examples_per_epoch = len(text)//(seq_length+1)

    # Create training examples / targets
    char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
    
    sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

    def split_input_target(chunk):
        input_text = chunk[:-1]
        target_text = chunk[1:]
        return input_text, target_text

    dataset = sequences.map(split_input_target)
    print()
    for input_example, target_example in  dataset.take(5):
        print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
        print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))
    print()
    # Batch size
    BATCH_SIZE = 64

    # Buffer size to shuffle the dataset
    # (TF data is designed to work with possibly infinite sequences,
    # so it doesn't attempt to shuffle the entire sequence in memory. Instead,
    # it maintains a buffer in which it shuffles elements).
    BUFFER_SIZE = 10000

    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
    print(dataset)

    # Length of the vocabulary in chars
    vocab_size = len(vocab)

    # The embedding dimension
    embedding_dim = 256

    # Number of RNN units
    rnn_units = 1024

    model = build_model_func(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)
    model.compile(optimizer='adam', loss=loss)
    
    for input_example_batch, target_example_batch in dataset.take(1):
        example_batch_predictions = model(input_example_batch)
        print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")    
    
    history = model.fit(dataset, epochs=EPOCHS, callbacks=callbacks)
    
    char2idx_str = json.dumps(char2idx)
    
    tfjs.converters.save_keras_model(model, path_to_save_model)
    
    with open(path_to_save_model+"char_idx_converter.js","w") as f:
        f.write("var char2idx ="+ str(char2idx)+"; \n var idx2char = "+str(idx2char.tolist()))
    
    tf.train.latest_checkpoint(checkpoint_dir)
    test_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
    test_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
    test_model.build(tf.TensorShape([1, None]))
    
    return model, test_model, char2idx, idx2char

In [0]:
def get_callbacks(checkpoint_dir):
    # Name of the checkpoint files
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

    checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_best_only=True,
    monitor='loss'
    )
    return [checkpoint_callback]

## Model Per Lyricists

In [0]:
def create_model_for(model_name, data_df):
    path_to_store = "./drive/My Drive/ColabDatasets/trained_models_v2/"+model_name+'/'
    checkpoint_dir = './'+model_name+'/training_checkpoints'

    model, test_model, char2idx, idx2char = create_model(
        model_name, 
        data_df, 
        path_to_store, 
        get_callbacks(checkpoint_dir),
        build_model,
        checkpoint_dir)
    results_df = evaluate_model(test_model, model_name, char2idx, idx2char, debug=True)
    results_df.to_csv(path_to_store+"result.csv")
    return results_df

In [27]:
results_vairamuthu = create_model_for("vairamuthu_songs", variamuthu_df)
results_vairamuthu

Creating Model:  vairamuthu_songs
-------------------------------
Total songs combined : 542

Length of text: 749212 characters
Sample Text: 

சிக்கு புக்கு சிக்கு புக்கு ரயிலே அட கலக்குது பார் இவ ஸ்டைலு
சிக்குவாளா சிக்குவாளா மயிலு இவ ஓக்கேன்னா அடி தூளு

சிக்கு புக்கு சிக்கு புக்கு ரயிலே அட கலக்குது பார் இவ ஸ்டைலு
சிக்குவாளா சிக்குவாளா மயிலு இவ ஓக்கேன்னா அடி தூளு

சின்ன பொண்ணிவ படிப்ப

133 unique characters

Input data:  'சிக்கு புக்கு சிக்கு புக்கு ரயிலே அட கலக்குது பார் இவ ஸ்டைலு\r\nசிக்குவாளா சிக்குவாளா மயிலு இவ ஓக்கேன்'
Target data: 'ிக்கு புக்கு சிக்கு புக்கு ரயிலே அட கலக்குது பார் இவ ஸ்டைலு\r\nசிக்குவாளா சிக்குவாளா மயிலு இவ ஓக்கேன்ன'
Input data:  'ா அடி தூளு\r\n\r\nசிக்கு புக்கு சிக்கு புக்கு ரயிலே அட கலக்குது பார் இவ ஸ்டைலு\r\nசிக்குவாளா சிக்குவாளா மய'
Target data: ' அடி தூளு\r\n\r\nசிக்கு புக்கு சிக்கு புக்கு ரயிலே அட கலக்குது பார் இவ ஸ்டைலு\r\nசிக்குவாளா சிக்குவாளா மயி'
Input data:  'லு இவ ஓக்கேன்னா அடி தூளு\r\n\r\nசின்ன பொண்ணிவ படிப்பது\xa0எத்திராஜா\xa0மனசையெல்லாம் சலவை செய்ய

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: generator 'ngrams' raised StopIteration
  


Model vairamuthu_songs - Diversity 0.1 - Uni score 1.0 - Bi score 0.907
Model vairamuthu_songs - Diversity 0.2 - Uni score 0.956 - Bi score 0.808
Model vairamuthu_songs - Diversity 0.30000000000000004 - Uni score 1.0 - Bi score 0.86
Model vairamuthu_songs - Diversity 0.4 - Uni score 0.957 - Bi score 0.833
Model vairamuthu_songs - Diversity 0.5 - Uni score 0.946 - Bi score 0.764
Model vairamuthu_songs - Diversity 0.6 - Uni score 0.972 - Bi score 0.868
Model vairamuthu_songs - Diversity 0.7000000000000001 - Uni score 0.973 - Bi score 0.661
Model vairamuthu_songs - Diversity 0.8 - Uni score 0.952 - Bi score 0.826
Model vairamuthu_songs - Diversity 0.9 - Uni score 0.91 - Bi score 0.607


,vairamuthu_songs_uni_score,vairamuthu_songs_bi_score,vairamuthu_songs_generated_lyric,vairamuthu_songs_unmatched_words
0,1.000,0.907,யாரோ\r\nஅலை வந்தாயே செந்தேனே...\r\n\r\nபார்க்க...,"[அலை வந்தாயே, தள்ளாதே உள்ளத்தை, கிளியே நிலவு, ..."
1,0.956,0.808,யாரோ\r\nஅலை வந்தாயே செந்தேனே...\r\n\r\nபார்க்க...,"[பிண்டேனே, கொல்லுக்கு, கவியாய்ச, அலை வந்தாயே,..."
2,1.000,0.860,யாரோ\r\nஅலை போல் எங்கும் மிதப்பேன்\r\n\r\nமார்...,"[போல் எங்கும், குட்டி தானே, தானே ஒரு, ஒரு கேளட..."
3,0.957,0.833,யாரோ\r\nநம்ம ஊரும் கொஞ்சம் கூசியிலே\r\n\r\nஅலை...,"[கூசியிலே, வெள்ளைப்பூவது, சிலுண்டு, கொஞ்சம் கூ..."
4,0.946,0.764,யாரோ\r\nஉலக நெனைக்கிறது\r\nகண்களிலே பெது நெஞ்ச...,"[நெனைக்கிறது, பெது, வளக்கி, பொறந்ததுன், உலக நெ..."
5,0.972,0.868,யாரோ\r\nஆற்று வீசும் போது கூட வேணாங்கோ\r\nஊனம்...,"[ஆளான், கண்கள்லை, ஆற்று வீசும், ஊர் பேர, வரிசை..."
6,0.973,0.661,யாரோ\r\nஅழகே சந்திரன்கள் விழி பூட்டு கொஞ்சிடும...,"[சந்திரன்கள், ஓளம், அழகே சந்திரன்கள், சந்திரன்..."
7,0.952,0.826,யாரோ (2)\r\n\r\nமுகந்தானை என் காதலா\r\n\r\nபேர...,"[முகந்தானை, பார்த்தவுடாதே, சாகயே, யாரோ (, முகந..."
8,0.910,0.607,யாரோ\r\nகரிச்சா நீ வீசிக்காத\r\n\r\n(ஆண் : என்...,"[கரிச்சா, வீசிக்காத, விரியால், தாலிட்டும், காந..."


In [28]:
results_vaali = create_model_for("vaali_songs", vaali_df)
results_vaali

Creating Model:  vaali_songs
-------------------------------
Total songs combined : 493

Length of text: 669906 characters
Sample Text: 

சிரிக்கின்றாள் இன்று சிரிக்கின்றாள்
சிந்திய கண்ணீர் மாறியதாலே
சிரிக்கின்றாள் இன்று சிரிக்கின்றாள்
சிந்திய கண்ணீர் மாறியதாலே

அன்புத் திருமுகம் காணாமல் -
நான்துன்பக் கடலில் நீந்தி வந்தேன்
காலப் புயலில் அணையாமல் -
நெஞ்சில்காதல் விளக்கை ஏந்தி வந

138 unique characters

Input data:  'சிரிக்கின்றாள் இன்று சிரிக்கின்றாள்\r\nசிந்திய கண்ணீர் மாறியதாலே\r\nசிரிக்கின்றாள் இன்று சிரிக்கின்றாள்\r'
Target data: 'ிரிக்கின்றாள் இன்று சிரிக்கின்றாள்\r\nசிந்திய கண்ணீர் மாறியதாலே\r\nசிரிக்கின்றாள் இன்று சிரிக்கின்றாள்\r\n'
Input data:  'சிந்திய கண்ணீர் மாறியதாலே\r\n\r\nஅன்புத் திருமுகம் காணாமல் -\r\nநான்துன்பக் கடலில் நீந்தி வந்தேன்\r\nகாலப் ப'
Target data: 'ிந்திய கண்ணீர் மாறியதாலே\r\n\r\nஅன்புத் திருமுகம் காணாமல் -\r\nநான்துன்பக் கடலில் நீந்தி வந்தேன்\r\nகாலப் பு'
Input data:  'யலில் அணையாமல் -\r\nநெஞ்சில்காதல் விளக்கை ஏந்தி வந்தேன்\r\nஉதய சூரியன் எதிரில் இருக்கையில்

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: generator 'ngrams' raised StopIteration
  


Model vaali_songs - Diversity 0.1 - Uni score 1.0 - Bi score 1.0
Model vaali_songs - Diversity 0.2 - Uni score 1.0 - Bi score 0.957
Model vaali_songs - Diversity 0.30000000000000004 - Uni score 1.0 - Bi score 0.979
Model vaali_songs - Diversity 0.4 - Uni score 1.0 - Bi score 0.894
Model vaali_songs - Diversity 0.5 - Uni score 0.986 - Bi score 0.927
Model vaali_songs - Diversity 0.6 - Uni score 1.0 - Bi score 0.926
Model vaali_songs - Diversity 0.7000000000000001 - Uni score 0.958 - Bi score 0.769
Model vaali_songs - Diversity 0.8 - Uni score 0.938 - Bi score 0.667
Model vaali_songs - Diversity 0.9 - Uni score 0.952 - Bi score 0.806


,vaali_songs_uni_score,vaali_songs_bi_score,vaali_songs_generated_lyric,vaali_songs_unmatched_words
0,1.000,1.000,யாரோ\r\n\r\nதாக்குதே\r\nகண் தாக்குதே\r\nகண் தா...,[]
1,1.000,0.957,யாரோ\r\n\r\nதாகம் தீர்ந்து வழங்கும் விருந்து\r...,"[தீர்ந்து வழங்கும், ஆசையில் அத]"
2,1.000,0.979,யாரோ\r\n\r\nதாகம் தீர்ந்து வழங்கும் விருந்து\r...,[தீர்ந்து வழங்கும்]
3,1.000,0.894,யாரோ\r\n\r\nதோகையும் தோகையும் தேகம் வாடுது\r\n...,"[ஆத்தாடி இல்லடி, இல்லடி கும்மி, கும்மி மோகம், ..."
4,0.986,0.927,யாரோ\r\nநான் செத்துப் பொழச்சவன்டா !\r\nஎமனை பா...,"[ஆனைகட்டியில், ஓடுகின்ற மன்னன், ஆனைகட்டியில் ச..."
5,1.000,0.926,யாரோ\r\n\r\nதென்றல் தேடுதே\r\n\r\nபுல்வெளி பாத...,"[தென்றல் தேடுதே, ஆடல்கள் பெண், கட்டிப்போடு வாழ..."
6,0.958,0.769,யாரோ\r\nநான் செத்துப் பொழச்சவன்டா !\r\nசெத்துப...,"[மூச்சிருப்புடம், தரிராதா, எரிய், புது அடிச்சா..."
7,0.938,0.667,யாரோ\r\n\r\nதேடி வேண்டும்\r\nஓர் பிள்ளை வந்தால...,"[குணி, அன்னம்தா, பெஞ்சான, நூர், அங்கிரு, தேடி ..."
8,0.952,0.806,யாரோ\r\nயார் நம் அசி\r\nஇந்த சொர்க்க தீவாழ்கவே...,"[அசி, தீவாழ்கவே, அன்ஹ, மாதினம், யார் நம், நம் ..."


In [29]:
results_kannadasan = create_model_for("kannadasan_songs", kannadasan_df)
results_kannadasan

Creating Model:  kannadasan_songs
-------------------------------
Total songs combined : 290

Length of text: 306758 characters
Sample Text: 

ஆறு முகமான பொருள் 
வான் மகிழ வந்தான்
அழகன் இவன் முருகன் எனும் 
இனிய பெயர் கொண்டான்

ஆறு முகமான பொருள் 
வான் மகிழ வந்தான்
அழகன் இவன் முருகன் எனும் 
இனிய பெயர் கொண்டான் ( இசை )
கால மகள் பெற்ற மகன் 
கோல முகம் வாழ்க
கால மகள் பெற்ற மகன் 
கோல ம

82 unique characters

Input data:  'ஆறு முகமான பொருள்\xa0\r\nவான் மகிழ வந்தான்\r\nஅழகன் இவன் முருகன் எனும்\xa0\r\nஇனிய பெயர் கொண்டான்\r\n\r\nஆறு முகமான '
Target data: 'று முகமான பொருள்\xa0\r\nவான் மகிழ வந்தான்\r\nஅழகன் இவன் முருகன் எனும்\xa0\r\nஇனிய பெயர் கொண்டான்\r\n\r\nஆறு முகமான ப'
Input data:  'ொருள்\xa0\r\nவான் மகிழ வந்தான்\r\nஅழகன் இவன் முருகன் எனும்\xa0\r\nஇனிய பெயர் கொண்டான் ( இசை )\r\nகால மகள் பெற்ற மக'
Target data: 'ருள்\xa0\r\nவான் மகிழ வந்தான்\r\nஅழகன் இவன் முருகன் எனும்\xa0\r\nஇனிய பெயர் கொண்டான் ( இசை )\r\nகால மகள் பெற்ற மகன'
Input data:  '்\xa0\r\nகோல முகம் வாழ்க\r\nகால மகள் பெற்ற மகன்\xa0\r\nகோ

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


Model kannadasan_songs - Diversity 0.1 - Uni score 1.0 - Bi score 1.0
Model kannadasan_songs - Diversity 0.2 - Uni score 1.0 - Bi score 1.0
Model kannadasan_songs - Diversity 0.30000000000000004 - Uni score 1.0 - Bi score 0.919


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: generator 'ngrams' raised StopIteration
  


Model kannadasan_songs - Diversity 0.4 - Uni score 1.0 - Bi score 0.94
Model kannadasan_songs - Diversity 0.5 - Uni score 0.939 - Bi score 0.62
Model kannadasan_songs - Diversity 0.6 - Uni score 0.969 - Bi score 0.75
Model kannadasan_songs - Diversity 0.7000000000000001 - Uni score 0.91 - Bi score 0.64
Model kannadasan_songs - Diversity 0.8 - Uni score 0.904 - Bi score 0.811
Model kannadasan_songs - Diversity 0.9 - Uni score 0.957 - Bi score 0.686


,kannadasan_songs_uni_score,kannadasan_songs_bi_score,kannadasan_songs_generated_lyric,kannadasan_songs_unmatched_words
0,1.000,1.000,யாரோ \r\nஅவன் பேர் தான் தெரியாதோ?\r\nயார் யார்...,[]
1,1.000,1.000,யாரோ \r\nஅவன் பேர் தான் தெரியாதோ?\r\nயார் யார்...,[]
2,1.000,0.919,யாரோ \r\nஅன்பே காவல் அவள் வழியாதோ\r\nதாயிருக்க...,"[அன்பே காவல், காவல் அவள், அவள் வழியாதோ]"
3,1.000,0.940,யாரோ \r\nஇன்று மட்டும் அமைதி தந்தால் உறங்குவேன...,"[.. எந்தன், எந்தன் பார்வை, இடையே சிவந்திருக்க]"
4,0.939,0.620,யாரோ \r\nஅன்பே காவல் அவள் வாழ்வும் முன்னே\r\nக...,"[மீணும், மேளி, பலர்களைக், இறுத்திருக்கும், அன்..."
5,0.969,0.750,யாரோ \r\n\r\nஅக்கு முகம் ஒன்று\r\nநூறு முகம் க...,"[இருந்ததில், படரட்டு, அக்கு முகம், தோள் நடு, ந..."
6,0.910,0.640,யாரோ\r\nஅன்பே காய் அரியது கண்ணில் தவம் எதற்கு\...,"[மலயே, பேரடியா, எண்ணெயில்லாத, உண்மைக்கொண்டு, த..."
7,0.904,0.811,யாரோ\r\nஅன்பே காவிரம் பாய் பீசாது\r\nது இரவினி...,"[காவிரம், பீசாது, கவித்தா, நிருப்பரங்குன்றம், ..."
8,0.957,0.686,யாரோ\r\n(பாட்டுப் போது வீகம் ஆசை தரும் மொழி கூ...,"[பார்க்கின்றாய், பொள்ளிருந்து, வாழ்த்துவந்தது,..."


In [30]:
results_all_songs = create_model_for("full_songs", df)
results_all_songs

Creating Model:  full_songs
-------------------------------
Total songs combined : 4142

Length of text: 5266494 characters
Sample Text: 

கடலினில் மீனாக இருந்தவள் நான்
உனக்கென கரை தாண்டி வந்தவள் தான்
துடித்திருந்தேன் தரையினிலே
திரும்பிவிட்டேன் என் கடலிடமே…

ஒரு நாள் சிரித்தேன்
மறு நாள் வெறுத்தேன்
உனை நான் கொல்லாமல்
கொன்று புதைத்தேனே
மன்னிப்பாயா மன்னிப்பாயா
மன்னிப்பாயா…

ஒரு

172 unique characters

Input data:  'கடலினில் மீனாக இருந்தவள் நான்\r\nஉனக்கென கரை தாண்டி வந்தவள் தான்\r\nதுடித்திருந்தேன் தரையினிலே\r\nதிரும்பி'
Target data: 'டலினில் மீனாக இருந்தவள் நான்\r\nஉனக்கென கரை தாண்டி வந்தவள் தான்\r\nதுடித்திருந்தேன் தரையினிலே\r\nதிரும்பிவ'
Input data:  'ிட்டேன் என் கடலிடமே…\r\n\r\nஒரு நாள் சிரித்தேன்\r\nமறு நாள் வெறுத்தேன்\r\nஉனை நான் கொல்லாமல்\r\nகொன்று புதைத்த'
Target data: 'ட்டேன் என் கடலிடமே…\r\n\r\nஒரு நாள் சிரித்தேன்\r\nமறு நாள் வெறுத்தேன்\r\nஉனை நான் கொல்லாமல்\r\nகொன்று புதைத்தே'
Input data:  'னே\r\nமன்னிப்பாயா மன்னிப்பாயா\r\nமன்னிப்பாயா…\r\n\r\nஒரு நாள் சிரித்தேன்\r\nமறு நாள் வெற

/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)
/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8

Model full_songs - Diversity 0.1 - Uni score 0.973 - Bi score 0.627
Model full_songs - Diversity 0.1 - Uni score 0.973 - Bi score 0.627
Model full_songs - Diversity 0.2 - Uni score 0.957 - Bi score 0.519
Model full_songs - Diversity 0.2 - Uni score 0.957 - Bi score 0.519
Model full_songs - Diversity 0.30000000000000004 - Uni score 0.974 - Bi score 0.786
Model full_songs - Diversity 0.30000000000000004 - Uni score 0.974 - Bi score 0.786
Model full_songs - Diversity 0.4 - Uni score 0.972 - Bi score 0.538
Model full_songs - Diversity 0.4 - Uni score 0.972 - Bi score 0.538
Model full_songs - Diversity 0.5 - Uni score 0.986 - Bi score 0.816
Model full_songs - Diversity 0.5 - Uni score 0.986 - Bi score 0.816
Model full_songs - Diversity 0.6 - Uni score 0.987 - Bi score 0.439
Model full_songs - Diversity 0.6 - Uni score 0.987 - Bi score 0.439
Model full_songs - Diversity 0.7000000000000001 - Uni score 0.884 - Bi score 0.62
Model full_songs - Diversity 0.7000000000000001 - Uni score 0.884 - Bi

,full_songs_uni_score,full_songs_bi_score,full_songs_generated_lyric,full_songs_unmatched_words
0,0.973,0.627,யாரோ\r\nகல்யாணம் கச்சேரி எப்போது தீரும்\r\n\r\...,"[பசித்தார், திரியே, காக்கும் காயம், காயம் என்ன..."
1,0.957,0.519,யாரோ\r\nகல்யாணம் கச்சேரி எப்போது தீரும்\r\n\r\...,"[கறைந்து, குடியில், பசியேறி, காக்கும் காய்ச்சல..."
2,0.974,0.786,யாரோ\r\nகல்யாணம் முடிவான ஆளான்னு\r\nசொன்னாலும்...,"[ஆளான்னு, ஒன்னோரு, கல்யாணம் முடிவான, முடிவான ஆ..."
3,0.972,0.538,யாரோ\r\nகல்யாணம் பண்ணாதே\r\nகாண்டா வான்னு கிப்...,"[போலீஸ்ஸு, லாப், கல்யாணம் பண்ணாதே, காண்டா வான்..."
4,0.986,0.816,யாரோ\r\nகலா ரசிகா ஹோய் கலா ரசிகா\r\nமஞ்சப்பட்ட...,"[மஞ்சப்பட்டி, மஞ்சப்பட்டி தாலி, சொல்லும் ராசா,..."
5,0.987,0.439,யாரோ\r\nகாலை மாலை தேவதை\r\nநான் தூங்க சென்று ப...,"[தேவாம, மாலை தேவதை, தூங்க சென்று, சென்று பூத்த..."
6,0.884,0.620,யாரோ\r\nதன் மோகம் முதல் காவியம் பெருமான்\r\nஅட...,"[இளமே, வெக்கிறவன், வெச்சிருக்கா, சல்லலோ, .லலலா..."
7,0.919,0.339,யாரோ (2)\r\nஇங்கு வருவதை பார்க்கும் போதும் உன்...,"[கொடிபூக்கள், வாங்கினாலும், முந்துக்குள், வந்த..."
8,0.863,0.232,யாரோ\r\nஅட இவ்வளவு கண்ட கனவா \r\nஎன் உச்சி மண்...,"[அதைய, என்னென்னம்மோ, கோடிப்பட்டி, சாதியான, அழை..."


,full_songs_uni_score,full_songs_bi_score,full_songs_generated_lyric,full_songs_unmatched_words
0,0.973,0.627,யாரோ\r\nகல்யாணம் கச்சேரி எப்போது தீரும்\r\n\r\...,"[பசித்தார், திரியே, காக்கும் காயம், காயம் என்ன..."
1,0.957,0.519,யாரோ\r\nகல்யாணம் கச்சேரி எப்போது தீரும்\r\n\r\...,"[கறைந்து, குடியில், பசியேறி, காக்கும் காய்ச்சல..."
2,0.974,0.786,யாரோ\r\nகல்யாணம் முடிவான ஆளான்னு\r\nசொன்னாலும்...,"[ஆளான்னு, ஒன்னோரு, கல்யாணம் முடிவான, முடிவான ஆ..."
3,0.972,0.538,யாரோ\r\nகல்யாணம் பண்ணாதே\r\nகாண்டா வான்னு கிப்...,"[போலீஸ்ஸு, லாப், கல்யாணம் பண்ணாதே, காண்டா வான்..."
4,0.986,0.816,யாரோ\r\nகலா ரசிகா ஹோய் கலா ரசிகா\r\nமஞ்சப்பட்ட...,"[மஞ்சப்பட்டி, மஞ்சப்பட்டி தாலி, சொல்லும் ராசா,..."
5,0.987,0.439,யாரோ\r\nகாலை மாலை தேவதை\r\nநான் தூங்க சென்று ப...,"[தேவாம, மாலை தேவதை, தூங்க சென்று, சென்று பூத்த..."
6,0.884,0.620,யாரோ\r\nதன் மோகம் முதல் காவியம் பெருமான்\r\nஅட...,"[இளமே, வெக்கிறவன், வெச்சிருக்கா, சல்லலோ, .லலலா..."
7,0.919,0.339,யாரோ (2)\r\nஇங்கு வருவதை பார்க்கும் போதும் உன்...,"[கொடிபூக்கள், வாங்கினாலும், முந்துக்குள், வந்த..."
8,0.863,0.232,யாரோ\r\nஅட இவ்வளவு கண்ட கனவா \r\nஎன் உச்சி மண்...,"[அதைய, என்னென்னம்மோ, கோடிப்பட்டி, சாதியான, அழை..."
